In [1]:
#pykrx 데이터 끌고오기
import pandas as pd
import numpy as np
import FinanceDataReader as fdr

In [8]:
data = pd.read_csv("../data/5. label 지정 완료.csv",encoding='cp949')

In [16]:
# 거래소 코드 6자리로 맞춰주기
for i in data.index:
    if len(str(data.loc[i,"거래소코드"])) == 3:
        data.loc[i,"거래소코드"]='000'+str(data.loc[i,"거래소코드"])
    elif len(str(data.loc[i,"거래소코드"])) == 4:
        data.loc[i,"거래소코드"]='00'+str(data.loc[i,"거래소코드"])
    elif len(str(data.loc[i,"거래소코드"])) == 5:
        data.loc[i,"거래소코드"]='0'+str(data.loc[i,"거래소코드"])
    elif len(str(data.loc[i,"거래소코드"])) == 2:
        data.loc[i,"거래소코드"]='0000'+str(data.loc[i,"거래소코드"])

#거래소 코드 개별로 추출
exchage_code =data["거래소코드"].unique()
exchage_code = pd.DataFrame(exchage_code)
exchage_code.columns=["거래소코드"]

#거래소코드와 회사명 확인
company_codes = data[["회사명","거래소코드"]]
codes_by_company = company_codes.groupby("회사명")["거래소코드"].unique()
codes_by_company = pd.DataFrame(codes_by_company)

#회사명 추출
company = data["회사명"].unique()
company = pd.DataFrame(company)
company.columns=["회사명"]

#데이터프레임 일치화
exchage_code["회사명"] = company["회사명"]
exchage_code

,거래소코드,회사명
0,058820,(주)CMG제약
1,050120,(주)ES큐브
2,079940,(주)가비아
3,217730,(주)강스템바이오텍
4,114190,(주)강원에너지
...,...,...
1369,192410,휴림네트웍스(주)
1370,090710,휴림로봇(주)
1371,205470,휴마시스(주)
1372,145020,휴젤(주)


In [17]:
fdr.DataReader("058820","2011-01-01", "2022-12-31")

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2011-01-03,1426,1440,1392,1394,439492,-0.023125
2011-01-04,1417,1417,1325,1329,671688,-0.046628
2011-01-05,1333,1351,1260,1321,470690,-0.006020
2011-01-06,1287,1333,1276,1333,422197,0.009084
2011-01-07,1300,1363,1287,1319,167534,-0.010503
...,...,...,...,...,...,...
2022-12-23,2225,2255,2160,2170,1044185,-0.046154
2022-12-26,2180,2230,2175,2185,782894,0.006912
2022-12-27,2185,2230,2180,2190,565804,0.002288


In [18]:
# 코스닥 개별기업들의 일자별 주가 추출
b=[]
for i in exchage_code.index:
    df = fdr.DataReader(str(exchage_code.loc[i,"거래소코드"]),"2011-01-01", "2021-12-31")
    b.append(df)

In [21]:
# 각 인덱스에 회사명이랑 거래소코드 넣어주기
for i in exchage_code.index:
    b[i]["거래소코드"]=exchage_code.loc[i,"거래소코드"]
    b[i]["회사명"] =exchage_code.loc[i,"회사명"]

# 기업별 데이터프레임 합하기
data_all = pd.concat(b).reset_index()
data_all 

,Date,Open,High,Low,Close,Volume,Change,거래소코드,회사명
0,2011-01-03,1426,1440,1392,1394,439492,-0.023125,058820,(주)CMG제약
1,2011-01-04,1417,1417,1325,1329,671688,-0.046628,058820,(주)CMG제약
2,2011-01-05,1333,1351,1260,1321,470690,-0.006020,058820,(주)CMG제약
3,2011-01-06,1287,1333,1276,1333,422197,0.009084,058820,(주)CMG제약
4,2011-01-07,1300,1363,1287,1319,167534,-0.010503,058820,(주)CMG제약
...,...,...,...,...,...,...,...,...,...
2899090,2021-12-24,7000,8450,6890,7550,10615180,0.083214,024060,흥구석유(주)
2899091,2021-12-27,7270,7290,6960,6980,1037631,-0.075497,024060,흥구석유(주)
2899092,2021-12-28,7080,7520,7060,7160,2100701,0.025788,024060,흥구석유(주)
2899093,2021-12-29,7160,7700,7100,7230,2085674,0.009777,024060,흥구석유(주)


In [22]:
data_all.to_csv("./data/1. 일일 주식 데이터.csv", encoding='cp949')

In [23]:
# 코스닥 데이터 추출
kosdaq = fdr.DataReader("KQ11","2011")  
kosdaq["수익률"] = kosdaq["Close"].pct_change()

C:\Users\dlsgm\AppData\Local\Temp\ipykernel_10204\2631656863.py:3: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  kosdaq["수익률"] = kosdaq["Close"].pct_change()


In [24]:
kosdaq.to_csv("./data/1. 코스닥 데이터.csv",encoding='cp949')